In [1]:
# Adapted from https://gist.github.com/papr/96f35bebcfea4e9b9552590868ca7f54
# See issue discussed on Pupil Labs' Discord: https://discord.com/channels/285728493612957698/285728493612957698/857998442999578664

import sys
sys.path.append("pupil_shared_modules/")

In [2]:
import json
import pathlib

import pandas as pd
import numpy as np

import file_methods as fm

Reads a pldata file and writes a new one into the `filtered`
subdirectory, for all data points for which `filter_fn` returns `True`

In [3]:
def filter_pldata(pldata_dir, pldata_topic, filter_fn):
    data = fm.load_pldata_file(pldata_dir, pldata_topic)
    pldata_dir /= "filtered"
    pldata_dir.mkdir(exist_ok=True)
    with fm.PLData_Writer(pldata_dir, pldata_topic) as writer:
        for d, ts, topic in zip(*data):
            if filter_fn(ts):
                writer.append_serialized(ts, topic, d.serialized)
        num_samples_removed = len(data.timestamps) - len(writer.ts_queue)
        print(
            f"\t{pldata_topic}: {num_samples_removed} samples removed outside recording range"
        )

Loop that looks for all recordings in `root_path`, and filters pupil and gaze data
such that samples from before the recording start are removed.

In [4]:
root_path = r'C:\Users\filipga\OneDrive - NTNU\Design ideation\Experiment\Data\Pupil data'
for info_json in sorted(pathlib.Path(root_path).glob("participant_0*/000/info.player.json")):
    with info_json.open() as info_fh:
        info_data = json.load(info_fh)
    start = info_data["start_time_synced_s"]
    duration = info_data["duration_s"] + 60 # add a generous buffer

    # Filter function. Only allow above start and below duration
    filter_fn = lambda ts: ts >= start and ts <= duration
    
    print(f"{info_json.parent} - {pd.to_datetime(start, unit='s')}")

    filter_pldata(info_json.parent, "pupil", filter_fn)
    filter_pldata(info_json.parent, "gaze", filter_fn)

C:\Users\filipga\OneDrive - NTNU\Design ideation\Experiment\Data\Pupil data\participant_001\000 - 1970-01-01 00:00:00.068950599
	pupil: 38 samples removed outside recording range
	gaze: 29 samples removed outside recording range
C:\Users\filipga\OneDrive - NTNU\Design ideation\Experiment\Data\Pupil data\participant_002\000 - 1970-01-01 00:00:00.046201599
	pupil: 40 samples removed outside recording range
	gaze: 30 samples removed outside recording range
C:\Users\filipga\OneDrive - NTNU\Design ideation\Experiment\Data\Pupil data\participant_003\000 - 1970-01-01 00:00:00.051737199
	pupil: 36 samples removed outside recording range
	gaze: 28 samples removed outside recording range
C:\Users\filipga\OneDrive - NTNU\Design ideation\Experiment\Data\Pupil data\participant_004\000 - 1970-01-01 00:00:00.047025399
	pupil: 32 samples removed outside recording range
	gaze: 26 samples removed outside recording range
C:\Users\filipga\OneDrive - NTNU\Design ideation\Experiment\Data\Pupil data\particip

Now you can make a backup of your original files and replace them with the filtered ones in your recording.
Afterward, you can open the recording in Player.